In [1]:
import import_ipynb
import pandas as pd
import numpy as np
import torch
from datetime import datetime
from datetime import datetime as dt
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import pickle
from threading import Thread
import threading
from IPython import display
import time,getopt,sys,os


from backtest import Backtest
from feeds import BackFeed,DataFeed
from validation import Validate
from add_features import add_features

from feeds import BackFeed,DataFeed
from featfuncs import feat_aug,add_addl_features_feed,add_ta_features_feed,add_sym_feature_feed
from featfuncs import add_global_indices_feed

from rlagents import RLStratAgentDyn, COLS, RLStratAgentDynFeatures
import warnings
warnings.filterwarnings('ignore')

importing Jupyter notebook from backtest.ipynb
importing Jupyter notebook from feeds.ipynb
importing Jupyter notebook from synfeed.ipynb
importing Jupyter notebook from india_calendar.ipynb
importing Jupyter notebook from featfuncs.ipynb
importing Jupyter notebook from feed_env.ipynb
importing Jupyter notebook from validation.ipynb
importing Jupyter notebook from rlagents.ipynb
importing Jupyter notebook from aiagentbase.ipynb


In [2]:
DATAPATH = '../algodata/realdata/alldata.csv'

In [3]:
def stringify(x):
    return pd.to_datetime(x['Datetime']).strftime('%d-%b-%Y')

In [4]:
DATAFILE=DATAPATH
print('Reading datafile')
df=pd.read_csv(DATAFILE)
if 'Date' not in df.columns: 
    print('Adding Date')
    df['Date']=df.apply(stringify,axis=1)
print('Creating feed')
feed=DataFeed(tickers=list(df.ticker.unique()[0:10]),dfgiven=True,df=df)
print('Processing feed')

add_addl_features_feed(feed,tickers=feed.tickers)
add_sym_feature_feed(feed,tickers=feed.tickers)

Reading datafile
Creating feed
Processing feed


In [5]:
RETAIN_OLD_FEATURES = True
ADD_FEATURES = False

In [6]:
with open(f'../algodata/realdata/datafeed_alldata_{RETAIN_OLD_FEATURES}_{ADD_FEATURES}.pkl','wb') as f: 
    pickle.dump(feed,f)

In [28]:
with open(f'../algodata/realdata/datafeed_alldata_{RETAIN_OLD_FEATURES}_{ADD_FEATURES}.pkl','rb') as f: 
    feed = pickle.load(f)

In [29]:
with open('additional_utils/cols.pkl', 'rb') as f:
        d = pickle.load(f)
        
# d['discrete_cols']+d['pre_discrete_cols']
imp_cols = d['imp_cols']
cols_to_use = d['cols_to_use']

In [30]:
from dataclasses import dataclass

@dataclass
class OptimalData:
    optD: dict
    optR: dict
    optDT: dict
    optRT: dict
    feed_original: pd.DataFrame
    feed_all: pd.DataFrame

In [31]:
import os
with open(os.path.join('additional_utils', 'optimal_alldata_new.pkl'), 'rb') as f:
    optimal_all_data = pickle.load(f)

In [34]:
feed_original = feed.data
for ticker in feed_original:
    feed_original[ticker]['row_num'] = feed_original[ticker]['row_num'].apply(lambda x: x if x<75 else x-75)
feed.data = feed_original

In [35]:
from copy import deepcopy
feed_old = deepcopy(feed)

In [36]:
feed = feed.data

In [37]:
optD = optimal_all_data.optD

for ticker in feed:
    feed[ticker]['Action'] = -5
    actions = []

    for t in optD[ticker]:
        for acts in optD[ticker][t]:
            rn, a = acts[0], acts[1]
            actions.append(a)
            # if a == -1: a = 2
            feed[ticker].loc[(feed[ticker]['Date']==t) & (feed[ticker]['row_num']==rn), 'Action'] = a

In [38]:
feed_original = pd.concat([feed[t] for t in feed], axis=0)
feed_original['Action'].value_counts()

-5    82772
-1      920
 1      764
Name: Action, dtype: int64

In [40]:
optDT = optimal_all_data.optDT

for ticker in feed:
    feed[ticker]['Action'] = -5
    actions = []

    for offset in optDT:
        for t in optDT[offset][ticker]:
            for acts in optDT[offset][ticker][t]:
                rn, a = acts[0], acts[1]
                actions.append(a)
                # if a == -1: a = 2
                feed[ticker].loc[(feed[ticker]['Date']==t) & (feed[ticker]['row_num']==rn), 'Action'] = a

In [41]:
feed_all = pd.concat([feed[t] for t in feed], axis=0)
feed_all['Action'].value_counts()

-5    81446
-1     1650
 1     1360
Name: Action, dtype: int64

In [42]:
optimal_all_data.feed_original = feed_original
optimal_all_data.feed_all = feed_all

In [43]:
import os
with open(os.path.join('additional_utils', 'optimal_alldata_new.pkl'), 'wb') as f:
    pickle.dump(optimal_all_data, f)